In [3]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import ttest_ind

dataset = pd.read_csv("AI-DATA-1 (1).xlsx - AI DATA.csv")
dataset.head()

,Unnamed: 0,Article no:,Unnamed: 0.1.1,authors,first_names,Genders (Gulce) Female:0 Male:1,Male Auhtors,Female Authors,Male auth,last_names,...,White,black,hispanic,asian,blank -,SUM,Countries.1,mistakes,white auth,Average income level
0,0.0,1,835.0,"['Jason D Morgenstern', 'Laura C Rosella', 'Ma...","['Jason D', 'Laura C', 'Mark J', 'Vivek', 'Hol...","['1', '0', '1', '1', '1', '1']",5,1,83%,"['Morgenstern', 'Rosella', 'Daley', 'Goel', 'S...",...,5,0,0,1,0,6.0,6.0,0.0,83%,NaN
1,1.0,2,1091.0,"['Monica I Lupei', 'Danni Li', 'Nicholas E Ing...","['Monica I', 'Danni', 'Nicholas E', 'Karyn D',...","['0', '0', '1', '0', '1', '1', '1', '0', '1', ...",7,4,64%,"['Lupei', 'Li', 'Ingraham', 'Baum', 'Benson', ...",...,10,0,0,1,0,11.0,11.0,0.0,91%,NaN
2,2.0,3,1250.0,"['Jack Fitzsimons', 'AbdulRahman Al Ali', 'Mic...","['Jack', 'AbdulRahman', 'Michael', 'Stephen']","['1', '1', '1', '1']",4,0,100%,"['Fitzsimons', 'Al Ali', 'Osborne', 'Roberts']",...,3,0,0,1,0,4.0,4.0,0.0,75%,NaN
3,4.0,5,800.0,"['Michelle S Mathis', 'Tosin E Badewa', 'Ruth ...","['Michelle S', 'Tosin E', 'Ruth N', 'Linda T',...","['0', '0', '0', '0', '1']",1,4,20%,"['Mathis', 'Badewa', 'Obiarinze', 'Wilkinson',...",...,2,1,0,0,2,5.0,5.0,0.0,40%,NaN
4,5.0,6,1072.0,"['Angier Allen', 'Samson Mataraso', 'Anna Sief...","['Angier', 'Samson', 'Anna', 'Hoyt', 'Gregory'...","['0', '1', '0', '1', '1', '1', '0', '0', '0', ...",6,7,46%,"['Allen', 'Mataraso', 'Siefkas', 'Burdick', 'B...",...,12,0,0,1,0,13.0,13.0,0.0,92%,NaN


In [122]:
research_papers = dataset[
    dataset["Research paper(1) or opinion piece? (0)"] == 1
].copy()

for count, row in research_papers.iterrows():
    if pd.isna(row["Last author - race  (asian, black, white, hispanic)"]):
        research_papers.at[
            count, "Last author - race  (asian, black, white, hispanic)"
        ] = row["First author - race  (asian, black, white, hispanic)"]
        research_papers.at[count, "Last author - gender (1 male, 0 female)"] = row[
            "First author - gender (1 male, 0 female)"
        ]
        research_papers.at[count, "Last author - country income level cf. User guide"] = row[
            "First author - country income level cf. User guide"
        ]


research_papers["Year"] = 2023 - research_papers["Year"]
research_papers["First author - white"] = (
    research_papers["First author - race  (asian, black, white, hispanic)"].str.lower()
    == "white"
)
research_papers["Last author - white"] = (
    research_papers["Last author - race  (asian, black, white, hispanic)"].str.lower()
    == "white"
)
research_papers["First author - male"] = (
    research_papers["First author - gender (1 male, 0 female)"] == 1
)
research_papers["Last author - male"] = (
    research_papers["Last author - gender (1 male, 0 female)"] == 1
)
research_papers["Funding"] = research_papers["Funding (yes =1, 0 = no)"] == 1
research_papers["Number of authors"] = research_papers["SUM"]

# converts from string to float between [0, 1]
research_papers["percent white auth"] = (
    research_papers["white auth"].apply(lambda x: x[:-1]).astype(float) / 100
)
research_papers["percent male auth"] = (
    research_papers["Male auth"].apply(lambda x: x[:-1]).astype(float) / 100
)

#Checks to make sure that no data is missing for the analysis of research papers
assert (
    research_papers[
        [
            "percent male auth",
            "percent white auth",
            "First author - white",
            "First author - male",
            "Last author - male",
            "Year",
            "Number of authors",
            "First author - country income level cf. User guide",
            "Last author - country income level cf. User guide",
        ]
    ]
    .isna()
    .sum()
    .sum()
    == 0
)

print(research_papers.shape)

(253, 150)


In [123]:
analysis_subset = research_papers[
    [
        "percent male auth",
        "percent white auth",
        "First author - white",
        "First author - male",
        "Last author - male",
        "Last author - white",
        "Funding",
        "Citations",
        "Year",
        "Number of authors",
        "Last author - country income level cf. User guide",
        "First author - country income level cf. User guide",
    ]
].copy()

In [124]:
analysis_subset.head()

,percent male auth,percent white auth,First author - white,First author - male,Last author - male,Last author - white,Funding,Citations,Year,Number of authors,Last author - country income level cf. User guide,First author - country income level cf. User guide
0,0.83,0.83,True,True,True,True,False,18.0,2.0,6.0,4.0,4.0
1,0.64,0.91,True,False,True,True,True,1.0,1.0,11.0,4.0,4.0
2,1.00,0.75,True,False,True,True,True,20.0,4.0,4.0,4.0,4.0
3,0.20,0.40,True,False,True,False,True,1.0,2.0,5.0,4.0,4.0
4,0.46,0.92,True,True,True,False,False,15.0,3.0,13.0,4.0,4.0


In [125]:
research_papers

,Unnamed: 0,Article no:,Unnamed: 0.1.1,authors,first_names,Genders (Gulce) Female:0 Male:1,Male Auhtors,Female Authors,Male auth,last_names,...,white auth,Average income level,First author - white,Last author - white,First author - male,Last author - male,Funding,Number of authors,percent white auth,percent male auth
0,0.0,1,835.0,"['Jason D Morgenstern', 'Laura C Rosella', 'Ma...","['Jason D', 'Laura C', 'Mark J', 'Vivek', 'Hol...","['1', '0', '1', '1', '1', '1']",5,1,83%,"['Morgenstern', 'Rosella', 'Daley', 'Goel', 'S...",...,83%,NaN,True,True,True,True,False,6.0,0.83,0.83
1,1.0,2,1091.0,"['Monica I Lupei', 'Danni Li', 'Nicholas E Ing...","['Monica I', 'Danni', 'Nicholas E', 'Karyn D',...","['0', '0', '1', '0', '1', '1', '1', '0', '1', ...",7,4,64%,"['Lupei', 'Li', 'Ingraham', 'Baum', 'Benson', ...",...,91%,NaN,True,True,False,True,True,11.0,0.91,0.64
2,2.0,3,1250.0,"['Jack Fitzsimons', 'AbdulRahman Al Ali', 'Mic...","['Jack', 'AbdulRahman', 'Michael', 'Stephen']","['1', '1', '1', '1']",4,0,100%,"['Fitzsimons', 'Al Ali', 'Osborne', 'Roberts']",...,75%,NaN,True,True,False,True,True,4.0,0.75,1.00
3,4.0,5,800.0,"['Michelle S Mathis', 'Tosin E Badewa', 'Ruth ...","['Michelle S', 'Tosin E', 'Ruth N', 'Linda T',...","['0', '0', '0', '0', '1']",1,4,20%,"['Mathis', 'Badewa', 'Obiarinze', 'Wilkinson',...",...,40%,NaN,True,False,False,True,True,5.0,0.40,0.20
4,5.0,6,1072.0,"['Angier Allen', 'Samson Mataraso', 'Anna Sief...","['Angier', 'Samson', 'Anna', 'Hoyt', 'Gregory'...","['0', '1', '0', '1', '1', '1', '0', '0', '0', ...",6,7,46%,"['Allen', 'Mataraso', 'Siefkas', 'Burdick', 'B...",...,92%,NaN,True,False,True,True,False,13.0,0.92,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,320.0,329,1125.0,"Brian E Dixon, Guoyu Tao, Jane Wang, Wanzhu Tu...",NaN,"1, 1, 0, 1, 0, 1, 0, 0",4,4,50%,NaN,...,50%,NaN,True,True,True,False,True,8.0,0.50,0.50
369,321.0,330,677.0,"['Ilias Maglogiannis', 'Elias Zafiropoulos', '...","['Ilias', 'Elias', 'Ioannis']","['1', '1', '1']",3,0,100%,"['Maglogiannis', 'Zafiropoulos', 'Anagnostopou...",...,100%,NaN,True,True,True,True,False,3.0,1.00,1.00
370,332.0,341 * do not have access,655.0,"Hyuk-Il Choi, Seok-Ki Jung, Seung-Hak Baek, Wo...",NaN,"1, 1, 1, 1, 1, 1, 1",7,0,100%,NaN,...,0%,NaN,False,False,True,True,False,7.0,0.00,1.00
371,360.0,369,56.0,"B.R. Kisby, M.M. McManus, I. Ponomarev\n",NaN,"1, 0, 0, 1",2,2,50%,NaN,...,100%,NaN,True,True,True,True,True,4.0,1.00,0.50


In [128]:
target = "Citations"
predictors = [
    "percent male auth",
    "percent white auth",
    "First author - white",
    "First author - male",
    "Last author - white",
    "Last author - male",
    "Year",
    "Number of authors",
]

ols = sm.OLS(
    analysis_subset[target].astype(float), analysis_subset[predictors].astype(float)
).fit()
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Citations   R-squared (uncentered):                   0.287
Model:                            OLS   Adj. R-squared (uncentered):              0.264
Method:                 Least Squares   F-statistic:                              12.33
Date:                Sun, 24 Sep 2023   Prob (F-statistic):                    8.17e-15
Time:                        18:07:43   Log-Likelihood:                         -1342.7
No. Observations:                 253   AIC:                                      2701.
Df Residuals:                     245   BIC:                                      2730.
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
percent male auth       -5.2626     13.841     -0.380      0.704     -32.526      22.001
percent white auth      -5.4907     18.604     -0.295      0.768     -42.136      31.155
First author - white    -0.9839     10.761     -0.091      0.927     -22.179      20.211
First author - male      4.2611      7.728      0.551      0.582     -10.961      19.483
Last author - white     -6.2623     10.376     -0.604      0.547     -26.700      14.175
Last author - male       8.6737      8.116      1.069      0.286      -7.313      24.660
Year                     3.7675      0.833      4.520      0.000       2.126       5.409
Number of authors        2.3170      0.691      3.354      0.001       0.956       3.678
==============================================================================
Omnibus:                      310.461   Durbin-Watson:                   1.838
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20517.151
Skew:                           5.287   Prob(JB):                         0.00
Kurtosis:                      45.831   Cond. No.                         57.1
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [129]:
target = "Funding"
predictors = [
    "percent male auth",
    "percent white auth",
    "First author - white",
    "First author - male",
    "Last author - white",
    "Last author - male",
    "Year",
    "Number of authors",
]

logit = sm.Logit(
    analysis_subset[target].astype(float), analysis_subset[predictors].astype(float)
).fit()
logit.summary()

Optimization terminated successfully.
         Current function value: 0.674477
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Funding   No. Observations:                  253
Model:                          Logit   Df Residuals:                      245
Method:                           MLE   Df Model:                            7
Date:                Sun, 24 Sep 2023   Pseudo R-squ.:                 0.02684
Time:                        18:07:44   Log-Likelihood:                -170.64
converged:                       True   LL-Null:                       -175.35
Covariance Type:            nonrobust   LLR p-value:                    0.2245
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
percent male auth       -0.2201      0.572     -0.385      0.700      -1.340       0.900
percent white auth       0.3402      0.760      0.447      0.655      -1.150       1.831
First author - white     0.1778      0.440      0.405      0.686      -0.684       1.039
First author - male      0.2559      0.317      0.808      0.419      -0.365       0.877
Last author - white      0.1884      0.423      0.446      0.656      -0.640       1.017
Last author - male      -0.3192      0.333     -0.958      0.338      -0.972       0.334
Year                    -0.0686      0.039     -1.780      0.075      -0.144       0.007
Number of authors        0.0030      0.028      0.105      0.916      -0.052       0.058
========================================================================================
"""

In [142]:
from scipy.stats import ttest_ind

def get_funded(df, group):
    df = df.astype(float)
    temp = df.groupby([group])["Funding"].describe()
    display(temp)
    group1 = df[df[group] == 0]["Funding"]
    group2 = df[df[group] == 1]["Funding"]
    print(ttest_ind(group1, group2))

get_funded(analysis_subset, "First author - male")

,count,mean,std,min,25%,50%,75%,max
First author - male,,,,,,,,
0.0,108.0,0.481481,0.501986,0.0,0.0,0.0,1.0,1.0
1.0,145.0,0.503448,0.501721,0.0,0.0,1.0,1.0,1.0


Ttest_indResult(statistic=-0.34438324424732025, pvalue=0.7308464454757725)


In [143]:
get_funded(analysis_subset, "First author - white")

,count,mean,std,min,25%,50%,75%,max
First author - white,,,,,,,,
0.0,103.0,0.436893,0.498427,0.0,0.0,0.0,1.0,1.0
1.0,150.0,0.533333,0.500559,0.0,0.0,1.0,1.0,1.0


Ttest_indResult(statistic=-1.5081964098177516, pvalue=0.1327619069439324)


In [146]:
get_funded(analysis_subset, "Last author - male")

,count,mean,std,min,25%,50%,75%,max
Last author - male,,,,,,,,
0.0,80.0,0.550000,0.500633,0.0,0.0,1.0,1.0,1.0
1.0,173.0,0.468208,0.500437,0.0,0.0,0.0,1.0,1.0


Ttest_indResult(statistic=1.2086915441012116, pvalue=0.2279187840281402)


In [147]:
get_funded(analysis_subset, "Last author - white")

,count,mean,std,min,25%,50%,75%,max
Last author - white,,,,,,,,
0.0,97.0,0.432990,0.498063,0.0,0.0,0.0,1.0,1.0
1.0,156.0,0.532051,0.500579,0.0,0.0,1.0,1.0,1.0


Ttest_indResult(statistic=-1.5333985679406883, pvalue=0.12643713669293302)
